# Import Modules

In [1]:
import os
import pandas as pd
from credentials import *
import snowflake.connector
from Queries.TPCH_SF1_ORDERS import *

c:\Users\ASANTE\miniconda3\envs\SNOWFLAKE\lib\site-packages\snowflake\connector\options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (12.0.1), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


# Helper Functions

In [2]:
def fetch_data(user: str, password: str, account: str, warehouse: str, database: str, schema: str, sql_query, col_names) -> pd.DataFrame:
    """
    Fetches data from a Snowflake database and returns it as a Pandas DataFrame.

    Parameters

    user: str
      name of snowflake user

    password: str
      password of snowflake user

    account: str
      id of account holder

    warehouse: str  
      name of snowflake warehouse
          
    database: str
      database name

    schema: str
      schema name

    sql_query: str
        query to execute

    col_names: list[str]
        column names to use

    Returns:
        result as pandas dataframe
    
    """
    try:
        conn = snowflake.connector.connect(user=user, password=password, account=account, warehouse=warehouse, database=database, schema=schema)
        results = conn.cursor().execute(f"{sql_query}").fetchall()
        print("Query executed successfully!")

        return pd.DataFrame(results, columns=col_names)
    
    except snowflake.connector.errors.Error as e:
        return f"Error executing query: {e}"

# Fetch Data

In [3]:
column_names = ['orderkey', 'custkey', 'orderstatus', 'totalprice', 'orderdate', 'orderpriority']

df = fetch_data(USERNAME, PASSWORD, ACCOUNT_ID, WAREHOUSE, DATABASE, SCHEMA, query1, column_names)

Query executed successfully!


# View DataFrame

In [4]:
df

,orderkey,custkey,orderstatus,totalprice,orderdate,orderpriority
0,600003,101318,F,188614.83,1992-11-27,4-NOT SPECIFIED
1,600069,47033,F,188365.87,1993-01-09,4-NOT SPECIFIED
2,600743,10183,F,172870.03,1994-01-22,2-HIGH
3,600099,64687,F,273339.09,1994-11-03,3-MEDIUM
4,600129,50956,O,139144.58,1997-01-08,2-HIGH
...,...,...,...,...,...,...
1499995,5399940,113116,O,153087.71,1996-12-23,4-NOT SPECIFIED
1499996,5399942,60880,O,108043.09,1995-05-08,4-NOT SPECIFIED
1499997,5399971,141763,F,73729.23,1992-07-13,4-NOT SPECIFIED
1499998,5399009,44168,F,47421.46,1994-09-02,2-HIGH
